In [1]:
from faker import Faker
import json
import sqlite3
import random
import secrets

In [2]:
# fetch the products id, names, price from the database
def fetch_products():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM pricings')
    pricings_table = c.fetchall()
    conn.close()
    return pricings_table

# fetch the customers id, names from the database
def fetch_customers():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM customers')
    customers_table = c.fetchall()
    conn.close()
    return customers_table

# fetch products from the database
def fetch_products_table():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT id, weight FROM products')
    pricings_table = c.fetchall()
    conn.close()
    return pricings_table

# from the product id look up the weight of the product
def get_product_weight(pricings_table, product_id):
    for product in pricings_table:
        if product[0] == product_id:
            return product[1]

# randomly select a customer id from the customers table
def random_customer_id(customers_table):
    return random.choice(customers_table)[0]

# from the customer id get the customer type
def get_customer_type(customers_table, customer_id):
    for customer in customers_table:
        if customer[0] == customer_id:
            return customer[4]
        
# from the customer id get the customer name, if it is a business customer then return the business name
def get_customer_name(customers_table, customer_id):
    for customer in customers_table:
        if customer[0] == customer_id:
            if customer[4] == "Business":
                return customer[3]
            else: # personal
                return f"{customer[1]} {customer[2]}"
        
# from the customer id get the customer address
def get_customer_address(customers_table, customer_id):
    for customer in customers_table:
        if customer[0] == customer_id:
            return customer[5]

# randomly select a product id from the pricings table
def random_product_id(pricings_table):
    return random.choice(pricings_table)[1]

# get the product name from the pricing table
def get_product_name(pricings_table, product_id):
    for product in pricings_table:
        if product[1] == product_id:
            return product[2]

# get the sku_units_ratio from the pricings table
def get_product_sku_units_ratio(pricings_table, product_id):
    for product in pricings_table:
        if product[1] == product_id:
            return product[3]

# get the product price from the pricing table based on the products id and type
def get_product_price(pricings_table, product_id, customer_type):
    for product in pricings_table:
        if product[1] == product_id and customer_type == "Business":
            return product[4]
        else : # personal
            return product[5]

# order id 36 hexes
def order_crypto_id():
    return secrets.token_hex(36)

# order quantity
def order_quantity():
    return random.randint(1, 15)

# order sku
def order_sku():
    sku = [
        'CARTON', # cartons
        'OUTER', # outer
        'BOTTLE', # bottle
        'PACKET', # packet
        'BUNDLE' # bundle
    ]
    
    weight = [
        0.1,
        0.25,
        0.2,
        0.35,
        0.1
    ]
    
    return random.choices(sku, weight,k=1)[0]

# order status
def order_status():
    status = [
        'pending',
        'processing',
        'completed',
        'cancelled'
    ]
    
    weight = [
        0.3,
        0.2,
        0.4,
        0.1
    ]
    
    return random.choices(status, weight, k=1)[0]

# order date
def order_date():
    fake = Faker()
    date_obj = fake.date_this_year()
    return date_obj.strftime('%Y-%m-%d')

In [3]:
def create_order_items(customer_type, order_id):
    products = fetch_products()
    products_table = fetch_products_table()
    product_id = random_product_id(products)
    product_name = get_product_name(products, product_id)
    amount = order_quantity()
    sku_type = order_sku()
    sku_units_ratio = get_product_sku_units_ratio(products, product_id)
        
    if sku_type == 'CARTON' or sku_type == 'BUNDLE':
        price = get_product_price(products, product_id, customer_type)
        total_price = price * amount
        weight = get_product_weight(products_table, product_id) * amount
    else:
        price = get_product_price(products, product_id, customer_type) * sku_units_ratio/100
        total_price = price * amount
        weight = get_product_weight(products_table, product_id) * amount * sku_units_ratio/100
    
    
    
    order_item = {
        'id': secrets.token_hex(36),
        "order_id": order_id,
        "product_id": product_id,
        "name": product_name,
        "quantity": amount,
        "sku": sku_type,
        "price": price,
        "total_price": total_price,
        "weight": weight,
        "sku_units_ratio": sku_units_ratio
    }
    return order_item

In [4]:
def create_order():
    customers = fetch_customers()
    customer_id = random_customer_id(customers)
    customer_type = get_customer_type(customers, customer_id)
    shipping_address = get_customer_address(customers, customer_id)
    order_id = order_crypto_id()
    

    order = {
        'id': order_id,
        'date': order_date(),
        'status': order_status(),
        'total_price': 0,
        'customer_id': customer_id,
        'customer_name': get_customer_name(customers, customer_id),
        'customer_type': customer_type,
        'shipping_address': shipping_address,
        'billing_address': shipping_address
    }
    
    return order


In [5]:
def create_multiple_order_items(n, order):
    order_items = []
    total = 0
    for i in range(0,n):
        temp = create_order_items(order['customer_type'], order['id'])
        order_items.append(temp)
        total = total + temp['total_price']
    
    order['total_price'] = total
    return order_items

def create_multiple_orders(n):
    orders = [create_order() for i in range(n)]
    return orders

In [6]:
order = create_multiple_orders(100)

orderItems = []

for i in range(len(order)):
    random_number = random.randint(3, 36)
    temp = create_multiple_order_items(random_number, order[i])
    orderItems.extend(temp)

#print(json.dumps(order, indent=4))
#print(json.dumps(orderItems, indent=4))
order_json = json.dumps(order, indent=4)
orderItems_json = json.dumps(orderItems, indent=4)

# save the json data as json
with open('json_data/orders.json', 'w') as f:
    f.write(order_json)
    
with open('json_data/order_items.json', 'w') as f:
    f.write(orderItems_json)

In [ ]:
# insert the orders into the database
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS orders (
        id TEXT PRIMARY KEY NOT NULL,
        date DATE NOT NULL,
        status TEXT NOT NULL,
        total_price REAL NOT NULL,
        customer_id TEXT NOT NULL,
        customer_name TEXT NOT NULL,
        customer_type TEXT NOT NULL,
        shipping_address TEXT NOT NULL,
        billing_address TEXT NOT NULL,
        FOREIGN KEY (customer_id) REFERENCES customers (id)
    )
    '''
)

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS order_items (
        id TEXT PRIMARY KEY NOT NULL,
        order_id TEXT NOT NULL,
        product_id TEXT NOT NULL,
        name TEXT NOT NULL,
        quantity INTEGER NOT NULL,
        price REAL NOT NULL,
        total_price REAL NOT NULL,
        sku TEXT NOT NULL,
        weight REAL NOT NULL,
        sku_units_ratio REAL NOT NULL,
        FOREIGN KEY (order_id) REFERENCES orders (id),
        FOREIGN KEY (product_id) REFERENCES products (id)
    )
    '''
    )
    
for order in json.loads(order_json):
    c.execute(
        '''
        INSERT INTO orders (id, date, status, total_price, customer_id, customer_name, customer_type, shipping_address, billing_address)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            order['id'],
            order['date'],
            order['status'],
            order['total_price'],
            order['customer_id'],
            order['customer_name'],
            order['customer_type'],
            order['shipping_address'],
            order['billing_address']
        )
    )
    
for order_item in json.loads(orderItems_json):
    c.execute(
        '''
        INSERT INTO order_items (id, order_id, product_id, name, quantity, price, total_price, sku, weight, sku_units_ratio)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            order_item['id'],
            order_item['order_id'],
            order_item['product_id'],
            order_item['name'],
            order_item['quantity'],
            order_item['price'],
            order_item['total_price'],
            order_item['sku'],
            order_item['weight'],
            order_item['sku_units_ratio']
        )
    )
    
conn.commit()
conn.close()

CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT NOT NULL,
    phone_number TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    postal_code TEXT,
    country TEXT,
    date_of_registration DATE
);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    category_id INTEGER,
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
);

CREATE TABLE categories (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name TEXT NOT NULL
);

CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

CREATE TABLE payments (
    payment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    payment_method TEXT NOT NULL,
    payment_date DATE NOT NULL,
    amount DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE shipping (
    shipping_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    shipping_date DATE,
    delivery_date DATE,
    shipping_cost DECIMAL(10, 2),
    shipping_address TEXT NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    order_status TEXT NOT NULL,
    total_amount DECIMAL(10, 2) NOT NULL,
    payment_method TEXT NOT NULL,
    shipping_address TEXT NOT NULL,
    billing_address TEXT NOT NULL,
    shipping_cost DECIMAL(10, 2),
    discount DECIMAL(10, 2),
    tax DECIMAL(10, 2),
    notes TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

In [ ]:
#conn = sqlite3.connect('database/database.db')
#c = conn.cursor()
#c.execute('DROP TABLE orders')
#conn.commit()